In [16]:
import warnings
from sklearn.utils import compute_sample_weight


warnings.simplefilter('ignore')

In [17]:
import matplotlib.pyplot as plt
import numpy as np

from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels

def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax


In [19]:
from pathlib import Path

import pandas as pd


csc_prediction_data_file = Path('./data_files/CSCFull.csv')

csc_prediction_true_values = ['Yes', 'Pass']
csc_prediction_false_values = ['No', 'Fail']
csc_prediction_data_types = {
    'Application Year': 'category',
    'Quintile': 'category',
    'EngBin': 'category',
    'AveSciBin': 'category',
    'ALQLBin': 'category',
    'Province': 'category',
}
csc_prediction_na_values = {
    'CSCBin': '*',
}

csc_prediction_data_rows = [
    'FinAid',
    'Application Year',
    'EngBin',
    'AveSciBin',
    'ALQLBin',
    'Province',
    'Quintile',
    'AdMathAttempt',
]

csc_prediction_data = pd.read_csv(
    filepath_or_buffer=csc_prediction_data_file,
    dtype=csc_prediction_data_types,
    true_values=csc_prediction_true_values,
    false_values=csc_prediction_false_values,
    na_values=csc_prediction_na_values,
    keep_default_na=False,
)
csc_prediction_data = csc_prediction_data.astype({'CSCBin': 'bool'})
csc_prediction_data.dtypes

FinAid                  bool
Application Year    category
EngBin              category
AveSciBin           category
ALQLBin             category
Province            category
Quintile            category
CSCBin                  bool
AdMathAttempt           bool
dtype: object

In [20]:
from collections import defaultdict

from autosklearn.classification import AutoSklearnClassifier
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import accuracy_score, f1_score, matthews_corrcoef
from sklearn.preprocessing import LabelEncoder


label_encoded_cols = defaultdict(LabelEncoder)

training_data = csc_prediction_data.apply(
    lambda x: label_encoded_cols[x.name].fit_transform(x)
)

X = training_data[csc_prediction_data_rows]
y = training_data['CSCBin']

leave_one_out = LeaveOneGroupOut()

# Auto-sklearn

In [21]:
csc_prediction_data.groupby('CSCBin').size()

CSCBin
False     91
True     692
dtype: int64

In [22]:
%%time

auto_sklearn_metrics = {
    'accuracy': [],
    'f1': [],
    'mcc': [],
}

for train_index, test_index in leave_one_out.split(X, y, X['Application Year']):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    automl = AutoSklearnClassifier(
#         time_left_for_this_task=1200,
        n_jobs=4,
    )
    automl.fit(X_train, y_train)
    
    y_hat = automl.predict(X_test)
    
    auto_sklearn_metrics['accuracy'].append(accuracy_score(y_test, y_hat))
    auto_sklearn_metrics['f1'].append(f1_score(y_test, y_hat))
    auto_sklearn_metrics['mcc'].append(matthews_corrcoef(y_test, y_hat))
    # 
    # np.set_printoptions(precision=2)
    # 
    # # Plot non-normalized confusion matrix
    # plot_confusion_matrix(y_test, y_hat, classes=['Pass', 'Fail'],
    #                       title='Confusion matrix, without normalization')
    # 
    # # Plot normalized confusion matrix
    # plot_confusion_matrix(y_test, y_hat, classes=['Pass', 'Fail'], normalize=True,
    #                       title='Normalized confusion matrix')
    # 
    # plt.show()

print(auto_sklearn_metrics)

[WARNING] [2019-04-12 17:15:34,944:EnsembleBuilder(1791095845):c27be3da148d3e84f3c3cb6ed4c689b3] No models better than random - using Dummy Score!
[WARNING] [2019-04-12 17:15:35,011:EnsembleBuilder(1791095845):c27be3da148d3e84f3c3cb6ed4c689b3] No models better than random - using Dummy Score!
[WARNING] [2019-04-12 17:15:37,017:EnsembleBuilder(1791095845):c27be3da148d3e84f3c3cb6ed4c689b3] No models better than random - using Dummy Score!
[WARNING] [2019-04-12 17:15:40,850:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger
[WARNING] [2019-04-12 17:15:40,850:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger
[WARNING] [2019-04-12 17:16:13,438:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger
[WARNING] [2019-04-12 17:16:13,438:smac.intensification.intensification.Intensifier] Challenge

# Auto-sklearn - Sample Weights

In [23]:
sample_weights = compute_sample_weight('balanced', y)
sample_weights[:5]

array([0.56575145, 0.56575145, 0.56575145, 0.56575145, 0.56575145])

In [24]:
%%time

auto_sklearn_metrics = {
    'accuracy': [],
    'f1': [],
    'mcc': [],
}


for train_index, test_index in leave_one_out.split(X, y, X['Application Year']):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    test_weights = sample_weights[test_index]
    
    automl = AutoSklearnClassifier(
#         time_left_for_this_task=1200,
        n_jobs=4,
    )
    automl.fit(X_train, y_train)
    
    y_hat = automl.predict(X_test)
    
    auto_sklearn_metrics['accuracy'].append(
        accuracy_score(y_test, y_hat, sample_weight=test_weights),
    )
    auto_sklearn_metrics['f1'].append(f1_score(y_test, y_hat, sample_weight=test_weights))
    auto_sklearn_metrics['mcc'].append(
        matthews_corrcoef(y_test, y_hat, sample_weight=test_weights),
    )
    # 
    # np.set_printoptions(precision=2)
    # 
    # # Plot non-normalized confusion matrix
    # plot_confusion_matrix(y_test, y_hat, classes=['Pass', 'Fail'],
    #                       title='Confusion matrix, without normalization')
    # 
    # # Plot normalized confusion matrix
    # plot_confusion_matrix(y_test, y_hat, classes=['Pass', 'Fail'], normalize=True,
    #                       title='Normalized confusion matrix')
    # 
    # plt.show()

print(auto_sklearn_metrics)

[WARNING] [2019-04-13 03:16:38,582:EnsembleBuilder(1791095845):c27be3da148d3e84f3c3cb6ed4c689b3] No models better than random - using Dummy Score!
[WARNING] [2019-04-13 03:16:38,669:EnsembleBuilder(1791095845):c27be3da148d3e84f3c3cb6ed4c689b3] No models better than random - using Dummy Score!
[WARNING] [2019-04-13 03:16:44,417:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger
[WARNING] [2019-04-13 03:16:44,417:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger
[WARNING] [2019-04-13 03:34:40,120:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger
[WARNING] [2019-04-13 03:34:40,120:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger
[WARNING] [2019-04-13 04:16:48,044:EnsembleBuilder(1791095845):9eff06c1e84d4b2f5812d01

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [25]:
# from tpot import TPOTClassifier


# tpot_metrics = {
#     'accuracy': [],
#     'f1': [],
#     'mcc': [],
# }

# for train_index, test_index in leave_one_out.split(X, y, X['Application Year']):
#     X_train, X_test = X.iloc[train_index], X.iloc[test_index]
#     y_train, y_test = y.iloc[train_index], y.iloc[test_index]

#     tpot = TPOTClassifier(
#         generations=5, 
#         population_size=20, 
#         max_time_mins=20, 
#         n_jobs=4,
#         verbosity=2,
#     )
#     tpot.fit(X_train, y_train)
   
#     y_hat = tpot.predict(X_test)
    
#     application_year_label_id = X_train['Application Year'].iloc[0]
#     application_year_label = label_encoded_cols['Application Year'].inverse_transform(
#         [application_year_label_id],
#     )[0]
    
#     tpot.export(f'tpot_csc_pipeline_{application_year_label}.py')
    
#     tpot_metrics['accuracy'].append(accuracy_score(y_test, y_hat))
#     tpot_metrics['f1'].append(f1_score(y_test, y_hat))
    
# print(tpot_metrics)